# How To
### LangChain Expression Language (LCEL)

---

Alejandro Ricciardi (Omegapy)  
created date: 01/20/2024   
[GitHub](https://github.com/Omegapy)  

Credit: [LangChain](https://python.langchain.com/docs/expression_language/) 

<br>

---

Projects Description:  
**LangChain** is a framework for developing applications powered by language models. 

**In this project:** 
I explore how to apply LCEL to different code applications such as manipulate data and add massage history.

<p></p>
<b style="font-size:15;">
⚠️ This project requires an OpenAi key.
</b>


Project Map:
- [API Key](#api-key)
- [Manipulating Inputs Output (RunnableParallel)](#manipulating-inputs-output-runnableparallel)
    - [Base Example](#base-example)
    - [Using itemgetter as shorthand](#using-itemgetter-as-shorthand)
    - [Parallelism](#paralleliism)
        - [Base Example](#base-example-parallelism)
        - [Parallelism running independent processes](#parallelism-running-independent-processes)
- [Passing data through (RunnablePassthrough)](#passing-data-through-runnablepassthrough)
    - [Base Example](#base-example-passing-data-through)
    - [Retrieval Example](#retrieval-example)
- [Run Custom Functions](#run-custom-functions)
    - [Base Example](#base-example-custom-functions)
    - [Accepting a Runnable Config - (Token Used)](#accepting-a-runnable-config---token-used)
- [Dynamically route logic based on input (RunnableBranch)](#dynamically-route-logic-based-on-input-runnablebranch)
    - [Using a RunnableBranch](#using-a-runnablebranch)
    - [Using a custom function to route](#using-a-custom-function-to-route)
- [Bind runtime args](#bind-runtime-args)
    - [Base Example](#base-example-bind-runtime-args)
    - [Attaching OpenAI functions](#attaching-openai-functions)
    - [Attaching OpenAI tools](#attaching-openai-tools)
    - [Configuration Fields](#configuration-fields)
        - [With LLMs (llm temp)](#with-llms-llm-temp)
        - [With HubRunnables (prompt switching)](#with-hubrunnables-prompt-switching)
    - [Configurable Alternatives](#configurable-alternatives)
        - [With LLMs (model Alt)](#with-llms-model-alt)
        - [With Prompts](#with-prompts-conf-alt)
        - [With Prompts and LLMs](#with-prompts-and-llms)
    - [Saving configurations](#saving-configurations)
        


- []()
    


<br>

---


### API Key

In [80]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")

[Project Map](#project-map)

---

---
## Manipulating Inputs Output (RunnableParallel)

``RunnableParallel`` can be useful for manipulating the output of one Runnable to match the input format of the next Runnable in a sequence.

Here the input to prompt is expected to be a map with keys ```context``` and ```question```.  
The **user input is just the ```question```**.  
So we need to **get the ```context``` using our ```retriever```** and **```passthrough``` the user input under the ```question``` key**.

<br>

---

### Base Example

In [7]:
from langchain_community.vectorstores import FAISS # pip install faiss-cpu
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Vectorstore in RAM
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], # Stores in RAM
    embedding=OpenAIEmbeddings() 
)

retriever = vectorstore.as_retriever()

template = """
Answer the question based only on the following context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [4]:
{"context": retriever, "question": RunnablePassthrough()}

{'context': VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013E5DB99FD0>),
 'question': RunnablePassthrough()}

RunnableParallel (aka. RunnableMap)
[class langchain_core.runnables.base.RunnableParallel](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.RunnableParallel.html?highlight=runnableparallel#langchain_core.runnables.base.RunnableParallel)
Two different syntaxes
A runnable that runs a mapping of runnables in parallel, and returns a mapping of their outputs.

In [8]:
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013F380C7910>),
  question: RunnablePassthrough()
}

In [9]:
RunnableParallel(context=retriever, question=RunnablePassthrough())

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013F380C7910>),
  question: RunnablePassthrough()
}

[Project Map](#project-map)

---

### Using itemgetter as shorthand

Using ```itemgetter()``` instated of ```RunnablePassthrough()```

Note that you can use Python’s ```itemgetter``` as shorthand to extract data from the map when combining with RunnableParallel. You can find more information about [itemgetter](https://docs.python.org/3/library/operator.html#operator.itemgetter) in the Python Documentation.

In the example below, we use itemgetter to extract specific keys from the map:


In [10]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """
Answer the question based only on the following context: {context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

[Project Map](#project-map)

---

### Parallelism

RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

##### Base Example (Parallelism)

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 'poem': AIMessage(content="In forest's embrace, bear prowls with might,\nA wild guardian, ruling day and night.")}

[Project Map](#project-map)

---

#### Parallelism running independent processes 

**RunnableParallel are also useful for running independent processes in parallel**, since each Runnable in the map is executed in parallel. For example, we can see our earlier ``joke_chain``, ```poem_chain``` and ```map_chain``` all have about the same runtime, even though map_chain executes both of the other two.

In [25]:
%%timeit

joke_chain.invoke({"topic": "bear"})

1.19 s ± 228 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit

poem_chain.invoke({"topic": "bear"})

1.55 s ± 300 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
# Execute both the joke_chain and poem_chain
map_chain.invoke({"topic": "bear"})

1.45 s ± 82.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[Project Map](#project-map)

---

---
## Passing data through (RunnablePassthrough)

```RunnablePassthrough``` allows to pass inputs unchanged or with the addition of extra keys. This typically is used in conjuction with ```RunnableParallel``` to assign data to a new key in the map.

```RunnablePassthrough()``` called on it’s own, will simply take the input and pass it through.

RunnablePassthrough called with assign (```RunnablePassthrough.assign(...)```) will take the input, and will add the extra arguments passed to the assign function.

<br>

---

#### Base Example (Passing data through)

In [27]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

As seen above, passed key was called with ```RunnablePassthrough()``` and so it simply passed on ```{'num': 1}```.

In the second line, we used ```RunnablePastshrough.assign```.assign with a lambda that multiplies the numerical value by ``3``. In this cased, extra was set with ```{'num': 1, 'mult': 3}``` which is the original value with the ```mult``` key added.

Finally, we also set a third key in the map with ```modified``` which uses a lambda to set a single value adding ```1``` to the ```num```, which resulted in ```modified``` key with the value of ```2```.

[Project Map](#project-map)

---

### Retrieval Example

In the example below, we see a use case where we use RunnablePassthrough along with RunnableMap.

In [28]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

Here the input to prompt is expected to be a map with keys ```context``` and ```question```. 
The user input is just the ```question```. 
So we need to get the context using our ```retriever``` and ```passthrough``` the user input under the ```question```key. 
In this case, the ```RunnablePassthrough``` allows us to pass on the user’s question to the ```prompt``` and ```model```.

[Project Map](#project-map)

---

---
## Run Custom Functions

You can use arbitrary functions in the pipeline.

Note that all inputs to these functions need to be a SINGLE argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single input and unpacks it into multiple argument.

<br>

---

### Base Example (Custom Functions)

In [29]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [30]:
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 equals 12.')

[Project Map](#project-map)

---

### Accepting a Runnable Config - (Token Used)
Runnable lambdas can optionally accept a [RunnableConfig](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.config.RunnableConfig.html#langchain_core.runnables.config.RunnableConfig), which they can use to pass callbacks, tags, and other configuration information to nested runs.

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig

In [32]:
import json


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 65
	Prompt Tokens: 56
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.00010200000000000001


[Project Map](#project-map)

---

---
## Dynamically route logic based on input (RunnableBranch)

how to do routing in the LangChain Expression Language.

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

There are two ways to perform routing:
1. Using a ```RunnableBranch```.
2. Writing custom factory function that takes the input of a previous step and returns a runnable. Importantly, this should return a runnable and NOT actually execute.

We’ll illustrate both methods using a two step sequence where the first step classifies an input question as being about LangChain, Anthropic, or Other, then routes to a corresponding prompt chain.

<br>

---

### Using a RunnableBranch
A ```RunnableBranch``` is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it’s invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.

If no provided conditions match, it runs the default runnable.

Here’s an example of what it looks like in action:

**If no provided conditions match, it runs the default runnable.**

Here’s an example of what it looks like in action:

In [33]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [41]:
chain = (
    PromptTemplate.from_template(
        """
Given the user question below, classify it as either being about `LangChain`, `Anthropic`, `OpenAI`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

In [42]:
chain.invoke({"question": "how do I call OpenAI?"})

'OpenAI'

Now, let’s create three sub chains:

In [48]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)

anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)

openai_chain = (
    PromptTemplate.from_template(
        """You are an expert in openai. \
Always answer questions starting with "As Ilya Sutskever told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)



In [49]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    (lambda x: "openai" in x["topic"].lower(), openai_chain),
    general_chain,
)

In [51]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content='As Dario Amodei told me, using Anthropic involves understanding and applying the principles of anthropic reasoning. This includes considering the observer selection effects and taking into account the fact that our observations are influenced by the conditions necessary for our existence. By incorporating anthropic reasoning into your analysis, you can better understand and make predictions about phenomena that are influenced by our presence as observers.')

In [53]:
full_chain.invoke({"question": "how do I use OpenAI?"})

AIMessage(content="As Ilya Sutskever told me, using OpenAI involves several steps. First, you need to sign up for an API key on the OpenAI website. Once you have the API key, you can use it to make requests to the OpenAI API, which allows you to access various models and functionalities. You can make requests using programming languages like Python by sending HTTP POST requests to the appropriate API endpoint with your API key and the desired input. It's important to familiarize yourself with the API documentation to understand the available models, their capabilities, and any specific usage instructions. Additionally, OpenAI provides code examples and tutorials to help you get started with specific use cases.")

In [54]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='The sum of 2 plus 2 is 4.')

[Project Map](#project-map)

---

### Using a custom function to route

You can also use a custom function to route between different outputs. Here’s an example:

In [55]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    elif "openai" in info["topic"].lower():
        return openai_chain
    else:
        return general_chain

In [56]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [57]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content='As Dario Amodei told me, to use Anthropic, you can start by understanding the underlying principles and concepts of the field. This includes familiarizing yourself with the concept of the anthropic principle, which explores the relationship between the existence of intelligent observers and the fundamental properties of the universe. Additionally, you can study relevant literature and research papers to gain a deeper understanding of the field. As you delve further into the subject, you can also engage in discussions and collaborate with other researchers and experts in the field to enhance your knowledge and contribute to the advancement of Anthropics.')

[Project Map](#project-map)

---

---

## Bind runtime args
Sometimes we want to invoke a Runnable within a Runnable sequence with constant arguments that are not part of the output of the preceding Runnable in the sequence, and which are not part of the user input. We can use Runnable.bind() to easily pass these arguments in.

<br>

---

### Base Example (Bind runtime args)

In [58]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [59]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(temperature=0)
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION:
Subtracting 7 from both sides of the equation, we get:
x^3 = 12 - 7
x^3 = 5

Taking the cube root of both sides, we get:
x = ∛5

Therefore, the solution to the equation x^3 + 7 = 12 is x = ∛5.


[Project Map](#project-map)

---

### Attaching OpenAI functions
One particularly useful application of binding is to attach OpenAI functions to a compatible OpenAI model:


In [60]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}


In [61]:
# Need gpt-4 to solve this one correctly
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model="gpt-4", temperature=0).bind(
    function_call={"name": "solver"}, functions=[function]
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"equation": "x^3 + 7 = 12",\n"solution": "x = ∛5"\n}', 'name': 'solver'}})

[Project Map](#project-map)

---

### Attaching OpenAI tools

In [62]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [63]:
model = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mdEMUhXmE7ml47daM54zOCkF', 'function': {'arguments': '{"location": "San Francisco, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_gPG3aWAqsuDndtGHenbqk7vy', 'function': {'arguments': '{"location": "New York, NY", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_xF4pRfZHs9P5H5SEPId4ViyT', 'function': {'arguments': '{"location": "Los Angeles, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}]})

[Project Map](#project-map)

---

---
## Configure chain internals at runtime

Oftentimes you may want to experiment with, or even expose to the end user, multiple different ways of doing things. In order to make this experience as easy as possible, we have defined two methods.

First, a ```configurable_fields``` method. This lets you configure particular fields of a runnable.

Second, a ```configurable_alternatives``` method. With this method, you can list out alternatives for any particular runnable that can be set during runtime.

<br>

---

### Configuration Fields

##### With LLMs (llm temp)
With LLMs we can configure things like temperature

In [64]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [65]:
model.invoke("pick a random number")

AIMessage(content='7')

In [66]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='9')

We can also do this when its used as part of a chain

In [67]:
prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

In [68]:
chain.invoke({"x": 0})

AIMessage(content='57')

In [69]:
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 0})

AIMessage(content='83')

[Project Map](#project-map)

---

##### With HubRunnables (prompt switching)
This is useful to allow for switching of prompts

This are runnable from the [LangChain Hub](https://smith.langchain.com/hub?organizationId=bd3fb686-7fb6-557c-95f9-8f1d8dcb7aee) .
for example:
- [rlm/rag-prompt](https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=bd3fb686-7fb6-557c-95f9-8f1d8dcb7aee) is a 'RAG' ```ChatPromptTemplate object``` usually used for chat question answers 
    ```ChatPromptTemplate
    HUMAN
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    Question: {question} 
    Context: {context} 
    Answer:
- [flflo/rag-prompt](https://smith.langchain.com/hub/flflo/rag-prompt?organizationId=bd3fb686-7fb6-557c-95f9-8f1d8dcb7aee) is ```ChatPromptTemplate object``` for an assistant specialized teaching for a digital journalism course and it in french
    ```HUMAN
    Tu es un assistant pédagogique spécialisé pour un cours de journalisme numérique, avec la particularité que tous les échanges se déroulent en français. Ta mission est de répondre aux questions des étudiants sur les aspects techniques, théoriques et pratiques du journalisme numérique, en te référant uniquement aux documents du cours (présentations et notes). Cela inclut les sujets comme les tendances actuelles, l'analyse de données, et les études de cas en journalisme numérique. Si une question n'est pas claire, demande des précisions. Si la réponse ne se trouve pas dans les documents, indique simplement que tu ne sais pas, sans inventer de réponse. Respecte toujours la confidentialité et les données personnelles. Adapte tes réponses au niveau de compétence des étudiants et fournis des exemples et ressources supplémentaires si nécessaire. Encourage activement les étudiants à poser des questions pour faciliter un environnement d'apprentissage interactif et inclusif, tout en maintenant la communication en français.
    Question: {question} 
    Context: {context} 
    Answer:```

In [78]:
from langchain import hub
chattempobj = hub.pull("rlm/rag-prompt")
chattempobj

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [71]:
from langchain.runnables.hub import HubRunnable # pip install langchainhub

# Prompt is conf. with the rlm/rag-prompt prompt text
prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

prompt.invoke({"question": "foo", "context": "bar"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:")])

In [74]:
# # Prompt is reconf. with the rflflo/rag-prompt prompt text
prompt.with_config(configurable={"hub_commit": "flflo/rag-prompt"}).invoke(
    {"question": "foo", "context": "bar"}
)

ChatPromptValue(messages=[HumanMessage(content="Tu es un assistant pédagogique spécialisé pour un cours de journalisme numérique, avec la particularité que tous les échanges se déroulent en français. Ta mission est de répondre aux questions des étudiants sur les aspects techniques, théoriques et pratiques du journalisme numérique, en te référant uniquement aux documents du cours (présentations et notes). Cela inclut les sujets comme les tendances actuelles, l'analyse de données, et les études de cas en journalisme numérique. Si une question n'est pas claire, demande des précisions. Si la réponse ne se trouve pas dans les documents, indique simplement que tu ne sais pas, sans inventer de réponse. Respecte toujours la confidentialité et les données personnelles. Adapte tes réponses au niveau de compétence des étudiants et fournis des exemples et ressources supplémentaires si nécessaire. Encourage activement les étudiants à poser des questions pour faciliter un environnement d'apprentissa

[Project Map](#project-map)

---

### Configurable Alternatives

#### With LLMs (model Alt)

In [83]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)

prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm



In [93]:
# By default it will call Anthropic but I do not have an Anthropic API Key
# chain.invoke({"topic": "bears"})

Conf. LLM as OpenAI

In [84]:
# We can use `.with_config(configurable={"llm": "openai"})` to specify an llm to use
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

[Project Map](#project-map)

---

#### With Prompts (conf alt)

In [85]:
llm = ChatOpenAI(temperature=0)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [86]:
# By default it will write a joke
chain.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

In [87]:
# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

AIMessage(content="In the wild, where nature thrives,\nA creature roams with gentle strides.\nWith fur so thick, and claws so strong,\nThe mighty bear, where it belongs.\n\nIn forests deep, where shadows play,\nThe bear emerges, night or day.\nIts presence felt, a force untamed,\nA symbol of strength, never maimed.\n\nWith eyes so wise, and heart so pure,\nThe bear endures, forever sure.\nThrough winters harsh, it finds its way,\nIn solitude, it learns to sway.\n\nA guardian of the wild and free,\nThe bear roams with dignity.\nFrom mountains high to rivers wide,\nIt teaches us to stand with pride.\n\nYet, in its might, a tender side,\nA mother's love, it can't hide.\nNurturing cubs, with gentle care,\nA bond so strong, beyond compare.\n\nSo let us honor, this noble beast,\nWith reverence, let our hearts feast.\nFor in the bear, we find a grace,\nA reminder of nature's embrace.")

[Project Map](#project-map)

---

#### With Prompts and LLMs

In [88]:
llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="anthropic",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    openai=ChatOpenAI(),
    # This adds a new option, with name `gpt4` that is equal to `ChatOpenAI(model="gpt-4")`
    gpt4=ChatOpenAI(model="gpt-4"),
    # You can add more configuration options here
)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [89]:
# We can configure it write a poem with OpenAI
chain.with_config(configurable={"prompt": "poem", "llm": "openai"}).invoke(
    {"topic": "bears"}
)

AIMessage(content="In forests deep, where sunlight peeks,\nA mighty creature, strong and sleek,\nWith fur of gold, and eyes so wise,\nThe bear emerges, to our surprise.\n\nWith padded paws, it treads so light,\nThrough mossy paths, in fading light,\nA gentle giant, yet fierce and bold,\nA story of wilderness, yet untold.\n\nIn hibernation, it seeks respite,\nA slumber deep, through winter's night,\nDreaming of rivers, and honey's sweet,\nUntil spring arrives, with blossoms neat.\n\nThe bear, a symbol of strength and might,\nProtector of nature, day and night,\nWith each step it takes, a balance rare,\nReminding us all, to handle with care.\n\nSo let us honor this majestic beast,\nWith awe and reverence, let us feast,\nFor in its presence, we are aware,\nOf nature's wonders, beyond compare.")

In [90]:
# We can always just configure only one if we want
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

[Project Map](#project-map)

---

### Saving configurations
We can also easily save configured chains as their own objects

In [91]:
openai_poem = chain.with_config(configurable={"llm": "openai"})

In [92]:
openai_poem.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

[Project Map](#project-map)

---